<a href="https://colab.research.google.com/github/parthvshah/lbw-classification/blob/master/GNB_LBW_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gaussian Naive Bayes implementation on Low Birth Weight Data Set

In [0]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
style.use('ggplot')
import pandas as pd

Take input from the dataset

In [0]:
lbw=pd.read_csv('part_1_0.csv')

In [97]:
print(lbw)

     community  age  weight  history   Hb  IFA        BP  res  reslt
0            1   26      37        1  5.9    1  1.444444    1      0
1            1   21      42        1  9.2    1  1.375000    1      0
2            1   21      36        1  8.8    1  1.500000    1      0
3            1   21      36        1  9.2    1  2.125000    1      0
4            1   21      36        1  8.0    1  1.375000    1      0
..         ...  ...     ...      ...  ...  ...       ...  ...    ...
96           3   21      55        1  9.0    0  1.375000    1      1
97           3   24      39        6  8.4    0  1.500000    1      1
98           3   24      50        1  9.1    0  1.375000    1      1
99           1   24      38        1  9.1    0  1.696758    1      1
100          3   21      50        1  9.0    1  1.375000    1      1

[101 rows x 9 columns]


Drop columns like history which have zero variance

In [0]:
X = lbw.drop("reslt", axis=1)
X=X.drop("history",axis=1) #this gives variance zero, so better drop this

y = lbw["reslt"]

Normalize Data before its fed into the model

In [0]:
import pandas as pd
from sklearn import preprocessing
#normalize
x =X #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X= pd.DataFrame(x_scaled)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state=10)
# Good seed values - 10


In [101]:
print("X_train",X_train)

X_train            0         1         2         3    4         5    6
2   0.000000  0.190476  0.171429  0.568627  1.0  0.023669  0.0
1   0.000000  0.190476  0.342857  0.647059  1.0  0.013807  0.0
42  0.000000  0.333333  0.257143  0.588235  1.0  0.013807  0.0
61  0.000000  0.380952  0.285714  0.647059  1.0  0.013807  0.0
44  0.000000  0.333333  0.285714  0.588235  1.0  0.040552  0.0
..       ...       ...       ...       ...  ...       ...  ...
89  0.000000  0.428571  0.400000  0.627451  0.0  0.039192  0.0
28  0.000000  0.333333  0.285714  0.705882  1.0  0.040575  0.0
64  1.000000  0.190476  0.857143  0.980392  0.0  0.013807  0.0
15  0.666667  1.000000  0.085714  0.803922  0.0  0.029304  0.0
9   0.000000  0.238095  0.000000  0.607843  1.0  0.042363  0.0

[90 rows x 7 columns]


In [102]:
print("y_train",y_train)

y_train 2     0
1     0
42    1
61    1
44    1
     ..
89    1
28    1
64    1
15    0
9     0
Name: reslt, Length: 90, dtype: int64


Calculated Prior Probabilities for the classes

In [103]:
def get_count_unique_vals(labels):
  return dict(labels.value_counts())


def prior_prob(labels):
    counts=get_count_unique_vals(labels)
    number_of_instances=labels.count()
    print(counts.items())
    priors={(key,value/number_of_instances) for key, value in counts.items()}
    return priors
    
priors=prior_prob(y_train)
print("priors",priors)

dict_items([(1, 67), (0, 23)])
priors {(1, 0.7444444444444445), (0, 0.25555555555555554)}


Calculate Mean and Variance For all Features

In [0]:
import math

In [0]:
def calculate_mean(df):
  return df.mean()
def calculate_std_dev(df):
  return df.std()

def Calculate_Mean_and_Variance(X_train,y_train):
  mean_and_variance_for_class={}
  classes=y_train.unique()
  for everyclass in classes:
      filtered_training_set=X_train[(y_train==everyclass)]
      mean_and_variance=dict()
      for every_attribute in list(X_train.columns.values):
          particular_attribute=filtered_training_set[every_attribute]
          mean_and_variance[every_attribute]=[]
          mean_for_this_attribute=calculate_mean(particular_attribute)
          mean_and_variance[every_attribute].append(mean_for_this_attribute)
          std_dev_for_this_attribute=calculate_std_dev(particular_attribute)
          var_for_this_attribute=math.pow(std_dev_for_this_attribute,2)
          mean_and_variance[every_attribute].append(var_for_this_attribute)
      mean_and_variance_for_class[everyclass]=mean_and_variance
  return mean_and_variance_for_class


For every class and attribute, we keep track of mean and variance

In [0]:
dictionary=Calculate_Mean_and_Variance(X_train,y_train)

In [107]:
print("variance and mean for every class and attribute",(dictionary))

variance and mean for every class and attribute {0: {0: [0.36231884057971014, 0.16073781291172598], 1: [0.34368530020703936, 0.04802237100373745], 2: [0.16770186335403722, 0.010337984996370089], 3: [0.6095481670929241, 0.03173300630800255], 4: [0.6086956521739131, 0.24901185770750983], 5: [0.04103152473715805, 0.004371592945846985], 6: [0.043478260869565216, 0.04347826086956522]}, 1: {0: [0.3930348258706469, 0.1579978893411729], 1: [0.31840796019900464, 0.015887374096329312], 2: [0.4737739872068229, 0.047400105225265124], 3: [0.6283289435177059, 0.01908861202359111], 4: [0.7611940298507462, 0.18453188602442347], 5: [0.04143100273896787, 0.014363824922731961], 6: [0.11940298507462686, 0.10673903211216655]}}


In [0]:
import operator

NOW using PDF Equation  
Given a feature

In [0]:
def calculate_probability(x, mean,variance):
    return (1/(math.sqrt(2*math.pi*variance))) * math.exp(-(math.pow(x-mean,2)/(2*variance)))

In [0]:
def predict(X_test,mean_variance):
    predictions = {}
    for index, row in X_test.iterrows():
        results = {}
        for k, v in priors:
            p = 0 #probability of every feature
            for feature in list(X_test.columns.values):
                prob = calculate_probability(row[feature], mean_variance[
                    k][feature][0], mean_variance[k][feature][1])
                if prob > 0:
                    p += math.log(prob)
            results[k] = math.log(v) + p
        predictions[index] = max(results.items(), key=operator.itemgetter(1))[0] # assign class that has maximum probability
    return predictions

In [0]:

predictions=predict(X_test,dictionary)

In [112]:
print("predictions for data in test set",predictions)

predictions for data in test set {79: 1, 63: 1, 85: 1, 39: 1, 38: 1, 3: 0, 19: 0, 66: 1, 90: 1, 14: 1, 43: 1}


In [113]:
def acc(y_test,prediction):
  count=0
  for ind,row in y_test.iteritems():
    if row == prediction[ind]:
      count+=1
  return count/len(y_test)*100.0

accuracy=acc(y_test,predictions)
print("accuracy :",accuracy)

accuracy : 90.9090909090909


In [0]:
#done